In [2]:
import pandas as pd
import os

# tky

In [3]:
# tky
df_train = pd.read_csv('../../Foursquare-TKY/train_sample.csv')
df_val = pd.read_csv('../../Foursquare-TKY/valid_sample.csv')
df_test = pd.read_csv('../../Foursquare-TKY/test_sample.csv')

In [ ]:
# train_filter following GETNext

from tqdm import tqdm

def train_filter(train_df: pd.DataFrame) -> pd.DataFrame:
    """
    filter short trajectory following GETNext setting (default = 2)
    """
    filtered_rows = []

    for traj_id in tqdm(train_df['pseudo_session_trajectory_id'].unique()):
        traj_df = train_df[train_df['pseudo_session_trajectory_id'] == traj_id]
        poi_ids = traj_df['PoiId'].to_list()

        input_seq = [(poi_ids[i], 0) for i in range(len(poi_ids) - 1)]
        
        if len(input_seq) >= 2:
            filtered_rows.append(traj_df)

    df_filtered = pd.concat(filtered_rows, ignore_index=True)

    return df_filtered

In [8]:
df_train_filter = train_filter(df_train)

100%|██████████| 51661/51661 [00:32<00:00, 1599.39it/s]


In [ ]:
# valid and test filter following GETNext

from tqdm import tqdm

def val_test_filter(train_df: pd.DataFrame, test_df: pd.DataFrame) -> pd.DataFrame:
    """
    val and test filter
    1. user id only in train set
    2. poi id only in train set
    3. filter short trajectory
    """
    # user id and poi id from train before short trajectory filter
    train_user_ids = set(train_df['UserId'].unique())
    train_poi_ids = set(train_df['PoiId'].unique())

    # 1. user id filter
    df_filtered = test_df[test_df['UserId'].isin(train_user_ids)]

    # 2. poi id filter
    df_filtered = df_filtered[df_filtered['PoiId'].isin(train_poi_ids)]

    # 3. filter short trajectory following GETNext setting (default = 2)
    #traj_counts = df_filtered.groupby('trajectory_id')['PoiId'].transform('count')
    #df_filtered = df_filtered[traj_counts >= 2]

    # 3. filter short trajectory following GETNext setting (default = 2)
    filtered_rows = []

    for traj_id in tqdm(df_filtered['pseudo_session_trajectory_id'].unique()):
        traj_df = df_filtered[df_filtered['pseudo_session_trajectory_id'] == traj_id]
        poi_ids = traj_df['PoiId'].to_list()

        input_seq = [(poi_ids[i], 0) for i in range(len(poi_ids) - 1)]
        
        if len(input_seq) >= 2:
            filtered_rows.append(traj_df)

    df_filtered = pd.concat(filtered_rows, ignore_index=True)

    df_filtered = df_filtered.reset_index(drop=True)

    return df_filtered

In [ ]:
df_val_filter = val_test_filter(df_train, df_val)

  0%|          | 0/7174 [00:00<?, ?it/s]

100%|██████████| 7174/7174 [00:02<00:00, 2953.16it/s]


In [ ]:
df_test_filter = val_test_filter(df_train, df_test)

  0%|          | 0/7079 [00:00<?, ?it/s]

100%|██████████| 7079/7079 [00:02<00:00, 2911.47it/s]


In [ ]:
# rename columns

def rename_and_select_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    rename and select columns

    Args:
        df (pd.DataFrame): input data

    Returns:
        pd.DataFrame: processed data
    """
    df_rename = df.rename(
        columns={
            "UserId": "user_id",
            "PoiId": "location_id",
            "PoiCategoryId": "category",
            "PoiCategoryName": "name",
            "Latitude": "latitude",
            "Longitude": "longitude",
            "UTCTimeOffset": "started_at",
            "pseudo_session_trajectory_id": "trajectory_id"
        }
    )[
        ["user_id", "location_id", "category", "name", "latitude", "longitude", "started_at", "trajectory_id"]
    ]
    return df_rename

In [11]:
df_train_processed = rename_and_select_columns(df_train_filter)
df_val_processed = rename_and_select_columns(df_val_filter)
df_test_processed = rename_and_select_columns(df_test_filter)

In [4]:
os.makedirs("tky/", exist_ok=True)

In [ ]:
df_train_processed.to_csv('tky/train.csv', index=False)
df_val_processed.to_csv('tky/valid.csv', index=False)
df_test_processed.to_csv('tky/test.csv', index=False)